In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
declare or replace qry_str string;

set var qry_str = 
  "create or replace view metv_dbsql
  (
    date comment 'Date when Databricks received the request.',
    workspace_name comment 'The name of the workspace.',
    client_application comment 'Client application that ran the statement. For example: Databricks SQL, Tableau, and Power BI. Nulls and blanks are labled as Unknown',
    compute_type comment 'The type of compute (Warehouse, Serverless, etc.)',
    warehouse_name comment 'The name of the SQL warehouse.',
    warehouse_type comment 'The type of SQL warehouse. Possible values are CLASSIC, PRO, and SERVERLESS.',
    warehouse_size comment 'The cluster size of the SQL warehouse. Possible values are 2X_SMALL, X_SMALL, SMALL, MEDIUM, LARGE, X_LARGE, 2X_LARGE, 3X_LARGE, and 4X_LARGE.',
    query_count comment 'Count of queries.',
    query_duration_hrs_sum comment 'Sum of total execution time of the statement in hours (Excluding result fetch time).',
    query_duration_sec_p50 comment 'P50 of total execution time of the statement in seconds (Excluding result fetch time).',
    query_duration_sec_p90 comment 'P90 of total execution time of the statement in seconds (Excluding result fetch time).',
    query_duration_sec_p95 comment 'P95 of total execution time of the statement in seconds (Excluding result fetch time).'
  )
  with metrics
  language yaml
  as $$
  version: 0.1
  
  source: " || :catalog || '.' || :schema || ".vw_query_history
  
  joins:
    - name: calendar
      source: " || :catalog || '.' || :schema || ".vw_dim_calendar
      on: calendar.calendar_key = source.calendar_key

    - name: compute
      source: " || :catalog || '.' || :schema || ".vw_dim_compute
      on: compute.compute_key = source.compute_key

    - name: workspace
      source: " || :catalog || '.' || :schema || ".vw_dim_workspace
      on: workspace.workspace_key = source.workspace_key
  
  dimensions:
    - name: date
      expr: calendar.calendar_date
    - name: workspace_name
      expr: workspace.workspace_name
    - name: client_application
      expr: source.client_application
    - name: compute_type
      expr: compute.compute_type
    - name: warehouse_name
      expr: compute.warehouse_name
    - name: warehouse_type
      expr: compute.warehouse_type
    - name: warehouse_size
      expr: compute.warehouse_size

  measures:
    - name: query_count
      expr: count(statement_id)
    - name: query_duration_hrs_sum
      expr: sum(total_duration_ms / 1000) / 3600
    - name: query_duration_sec_p50
      expr: percentile(total_duration_ms / 1000, 0.5)
    - name: query_duration_sec_p90
      expr: percentile(total_duration_ms / 1000, 0.9)
    - name: query_duration_sec_p95
      expr: percentile(total_duration_ms / 1000, 0.95)
  $$";

execute immediate qry_str;